In [1]:
import numpy as np

timesteps = 100 # 입력 시퀀스에 있는 타임스텝의 수
input_features = 32 # 입력 특성의 차원
output_features = 64 # 출력 특성의 차원

inputs = np.random.random((timesteps, input_features)) # 입력 데이터: 예제를 위해 생성한 난수

state_t = np.zeros((output_features, )) # 초기 상태: 모두 0인 벡터

# 랜덤한 가중치 행렬 생성
W = np.random.random((output_features, input_features))
U = np.random.random((output_features, output_features))
b = np.random.random((output_features, ))

successive_outputs = []
for input_t in inputs:
    output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)
    
    successive_outputs.append(output_t)
    state_t = output_t
    
final_output_sequence = np.stack(successive_outputs, axis=0)

In [2]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32))
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________


In [3]:
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000 # 특성으로 사용할 단어의 수
maxlen = 500 # 사용할 텍스트의 길이(가장 빈번한 max_features개의 단어만 사용)
batch_size = 32

print('데이터 로딩...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)
print(len(input_train), '훈련 시퀀스')
print(len(input_test), '테스트 시퀀스')

print('시퀀스 패딩 (samples x time)')

input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print('input_train 크기:', input_train.shape)
print('input_test 크기:', input_test.shape)

데이터 로딩...
25000 훈련 시퀀스
25000 테스트 시퀀스
시퀀스 패딩 (samples x time)
input_train 크기: (25000, 500)
input_test 크기: (25000, 500)


In [4]:
from keras.layers import Dense

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(input_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 16s 825us/step - loss: 0.6443 - acc: 0.6117 - val_loss: 0.4845 - val_acc: 0.7832
Epoch 2/10
20000/20000 [==============================] - 15s 750us/step - loss: 0.4503 - acc: 0.8001 - val_loss: 0.4429 - val_acc: 0.8066
Epoch 3/10
20000/20000 [==============================] - 15s 751us/step - loss: 0.3016 - acc: 0.8791 - val_loss: 0.3426 - val_acc: 0.8510
Epoch 4/10
20000/20000 [==============================] - 15s 755us/step - loss: 0.2396 - acc: 0.9037 - val_loss: 0.3771 - val_acc: 0.8440
Epoch 5/10
20000/20000 [==============================] - 15s 752us/step - loss: 0.1843 - acc: 0.9301 - val_loss: 0.3990 - val_acc: 0.8600
Epoch 6/10
20000/20000 [==============================] - 15s 752us/step - loss: 0.1314 - acc: 0.9526 - val_loss: 0.4034 - val_acc: 0.8474
Epoch 7/10
20000/20000 [==============================] - 15s 749us/step - loss: 0.0857 - acc: 0.9714 - val_loss: 0.4